The `FibonacciEvent` class represents a single point (an event) in the 3D Minkowski spacetime.
- Each event corresponds to a specific Fibonacci number, identified by its `index` (e.g., 0 for F_0, 1 for F_1, 2 for F_2, etc.) and its actual numerical `value`.
- It has coordinates `t` (time), `x`, `y`, and `z` (spatial).
- `t` is initially set to the Fibonacci index but will be adjusted during the causality enforcement simulation.
- `x`, `y`, and `z` are initialized to random values between -1 and 1.
- The `get_coords` method returns the event's coordinates.
- The `__repr__` method provides a user-friendly string representation of the event.
- The `__lt__` method is implemented to allow `FibonacciEvent` objects to be compared based on their index, which is necessary for the `heapq` (priority queue) used in A\* search.

### 3. MinkowskiSpace Class
The `MinkowskiSpace` class manages a collection of `FibonacciEvent` objects.
- The `__init__` method initializes an empty dictionary `events` to store the events, keyed by their Fibonacci index.
- `add_event` and `get_event` are utility methods for managing events in the dictionary.
- `calculate_spacetime_interval_sq` computes the squared spacetime interval between two events using the Minkowski metric (not standard Euclidean distance). A negative or zero squared interval means the events are causally connected (timelike or lightlike).
- `is_causally_connected` checks if one event could causally influence another. This requires the potential parent to be earlier in time and the spacetime interval between them to be timelike or lightlike (within a small tolerance `epsilon_1`).
- `enforce_causality_step` is a simplified simulation of the paper's algorithm to adjust event times to ensure that parent events are always in the past light cone of their children events in the *embedded* spacetime. If a causal violation is found, the child event's time is moved forward.
- `find_parents_by_causality` simulates retrieving parent events by querying the past light cone of a child event within the embedded spacetime. It looks for events with smaller indices and checks if they are causally connected. It specifically checks for the expected Fibonacci parents (indices `N-1` and `N-2`).

### 4. Embedding Simulation
The `simulate_fibonacci_embedding` function orchestrates the process of embedding Fibonacci numbers.
- It creates a `MinkowskiSpace`.
- It generates a specified number of `FibonacciEvent` objects, one for each Fibonacci index, initializing their coordinates.
- It then enters an iterative loop (`num_enforcement_iterations`) to enforce the causal relationships. In each iteration, it checks pairs of events (specifically potential parents like F_{i-1} and F_{i-2} for a child F_i) and adjusts the child's time coordinate if the causal condition is violated.
- This iterative process aims to arrange the events in spacetime such that the inherent hierarchical structure of the Fibonacci sequence (where `F_N` depends on `F_{N-1}` and `F_{N-2}`) is reflected in their causal relationships in the embedding.
- The loop breaks early if an iteration completes without any adjustments, suggesting the embedding has reached a stable causal configuration.

### 5. Finding Next Fibonacci Randomly
This function `find_next_fibonacci_randomly` demonstrates how, once events are embedded and causality is enforced, you might "discover" the next Fibonacci number.
- It takes a `start_index` (the index of the current Fibonacci number, F_N).
- It retrieves the corresponding event from the `MinkowskiSpace`.
- It uses `find_parents_by_causality` to find the events in the past light cone that are considered its causal parents within the embedding.
- It then attempts to identify the F_{N-1} event among these parents and uses the core Fibonacci relationship (`F_{N+1} = F_N + F_{N-1}`) to calculate the next Fibonacci number. This simulates inferring the mathematical structure from the causal structure of the embedding.

### 6. A* Search Implementation
This function `a_star_search` implements the A\* pathfinding algorithm within the embedded Minkowski space.
- The "nodes" in this search are the Fibonacci indices.
- The possible "edges" represent movements forward in the Fibonacci sequence, specifically from index `N` to `N+1` or `N+2`.
- The goal is to find a path of indices from a `start_index` to a `goal_index`.
- The cost to move between indices (`g_score`) is simply `1` for each step.
- The heuristic function (`h_score`) estimates the remaining cost to reach the goal. Here, it uses the Euclidean spatial distance in the `(x, y, z)` coordinates between the current event and the goal event. While the paper focuses on causal structure (timelike/lightlike intervals), A\* typically uses a distance heuristic. Using the spatial distance here is a simplification for demonstration.
- `heapq` is used to manage the priority queue of nodes to explore, prioritizing nodes with a lower total estimated cost (`f_score = g_score + h_score`).
- If a path is found, it returns a list of Fibonacci indices from start to goal.

## Overall
In essence, this code provides a simulation of how the abstract mathematical relationships of the Fibonacci sequence might be represented and navigated within a geometric structure (Minkowski spacetime) by enforcing causal constraints derived from their dependencies (F_N depends on F_{N-1} and F_{N-2}). It's a conceptual demonstration of embedding mathematical structures into a spacetime geometry for potentially discovering or understanding their properties through spatial/causal exploration.

In [2]:
import math
import random
import heapq
import time


In [29]:
def get_fibonacci_number(n):
    """Calculates the nth Fibonacci number using an iterative method."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b

# --- 2. FibonacciEvent Class (Event in Minkowski Spacetime) ---
class FibonacciEvent:
    """Represents a Fibonacci number's index as an event in 3D Minkowski Spacetime."""
    def __init__(self, index, value, t=None, x=None, y=None, z=None):
        self.index = index
        self.value = value # The actual Fibonacci numerical value

        # Initialize time based on index (as per your idea)
        # The paper's algorithm adjusts 't' later.
        self.t = float(t) if t is not None else float(index)

        # Initialize spatial coordinates randomly (as per paper's initialization)
        self.x = float(x) if x is not None else random.uniform(-1, 1)
        self.y = float(y) if y is not None else random.uniform(-1, 1)
        self.z = float(z) if z is not None else random.uniform(-1, 1)

    def get_coords(self):
        return (self.t, self.x, self.y, self.z)

    def __repr__(self):
        return f"E({self.index}, F={self.value}, t={self.t:.2f}, x={self.x:.2f}, y={self.y:.2f}, z={self.z:.2f})"

    def __lt__(self, other):
        # For heapq comparison in A*
        return self.index < other.index

# --- 3. MinkowskiSpace Class ---
class MinkowskiSpace:
    """Manages Fibonacci events and their causal relationships."""
    def __init__(self):
        self.events = {} # Stores FibonacciEvent objects, keyed by index

    def add_event(self, event: FibonacciEvent):
        """Adds a Fibonacci event to the spacetime."""
        self.events[event.index] = event

    def get_event(self, index):
        """Retrieves an event by its Fibonacci index."""
        return self.events.get(index)

    def calculate_spacetime_interval_sq(self, event1: FibonacciEvent, event2: FibonacciEvent):
        """
        Calculates the squared spacetime interval (Delta s^2) between two events.
        Delta s^2 = -Delta t^2 + Delta x^2 + Delta y^2 + Delta z^2
        """
        dt = event1.t - event2.t
        dx = event1.x - event2.x
        dy = event1.y - event2.y
        dz = event1.z - event2.z

        # Minkowski metric: -dt^2 + dx^2 + dy^2 + dz^2
        return -dt**2 + dx**2 + dy**2 + dz**2

    def is_causally_connected(self, potential_parent: FibonacciEvent, child: FibonacciEvent, epsilon_1=1e-5):
        """
        Checks if potential_parent is in the past light cone of child.
        Based on paper's conditions:
        1. t_parent < t_child (temporal order)
        2. (t_child - t_parent)^2 > D_spatial^2 (timelike or lightlike interval)
        """
        if potential_parent.t >= child.t:
            return False # Parent must be in the past

        dt = child.t - potential_parent.t

        # Euclidean spatial distance squared
        dx = child.x - potential_parent.x
        dy = child.y - potential_parent.y
        dz = child.z - potential_parent.z
        d_spatial_sq = dx**2 + dy**2 + dz**2

        # Check if it's timelike or lightlike (within epsilon for "almost null")
        # (t_child - t_parent)^2 >= D_spatial^2
        return dt**2 >= d_spatial_sq - epsilon_1 # Add epsilon to allow for 'almost null'

    def enforce_causality_step(self, parent_event: FibonacciEvent, child_event: FibonacciEvent, epsilon_1=1e-5, epsilon_2=0):
        """
        Simulates one step of time adjustment to enforce causality, as per paper (Eq 3.2).
        This is a simplified, single-step adjustment, not a full convergence algorithm.
        Returns True if an adjustment was made, False otherwise.
        """
        dt_current = child_event.t - parent_event.t

        dx = child_event.x - parent_event.x
        dy = child_event.y - parent_event.y
        dz = child_event.z - parent_event.z
        d_spatial = math.sqrt(dx**2 + dy**2 + dz**2)

        violation = False
        if dt_current <= 0: # Parent not in past
            violation = True
        elif dt_current < d_spatial - epsilon_1: # Not timelike/lightlike enough
            violation = True

        if violation:
            t_min = d_spatial + epsilon_1
            delta = t_min - dt_current

            # Apply adjustments (epsilon_2=0 means only child's time is adjusted)
            # child_event.t += delta * (1 - epsilon_2) # Original paper
            child_event.t += delta # Simplified for epsilon_2 = 0
            # parent_event.t -= delta * epsilon_2 # Original paper
            return True
        return False

    def find_parents_by_causality(self, child_event: FibonacciEvent, max_parent_lookback=3):
        """
        Finds potential parents of a child event by querying its past light cone.
        Simulates the paper's retrieval mechanism.
        """
        potential_parents = []

        # Look at events with smaller indices (potential parents)
        for i in range(max(0, child_event.index - max_parent_lookback), child_event.index):
            parent_candidate = self.get_event(i)
            if parent_candidate and self.is_causally_connected(parent_candidate, child_event):
                # Calculate proper time (tau^2 = -Delta s^2) for minimal proper time selection
                ds_sq = self.calculate_spacetime_interval_sq(parent_candidate, child_event)
                if ds_sq <= 0: # Ensure it's timelike or lightlike
                    proper_time_sq = -ds_sq
                    potential_parents.append((proper_time_sq, parent_candidate))

        # Sort by proper time (smallest proper time first) and return the actual Fibonacci parents
        potential_parents.sort(key=lambda x: x[0])

        # In the Fibonacci sequence, parents are F_N-1 and F_N-2.
        # We assume the embedding algorithm would have correctly positioned them.
        # Here, we just pick the ones with the correct indices if they are causally connected.
        actual_fib_parents = []
        if child_event.index >= 1:
            parent1_idx = child_event.index - 1
            parent1_event = self.get_event(parent1_idx)
            if parent1_event and self.is_causally_connected(parent1_event, child_event):
                actual_fib_parents.append(parent1_event)

        if child_event.index >= 2:
            parent2_idx = child_event.index - 2
            parent2_event = self.get_event(parent2_idx)
            if parent2_event and self.is_causally_connected(parent2_event, child_event):
                actual_fib_parents.append(parent2_event)

        return actual_fib_parents

# --- 4. Explorer and Embedding Simulation ---
def simulate_fibonacci_embedding(num_fib_events, num_enforcement_iterations=500):
    """
    Simulates creating Fibonacci events and enforcing causal hierarchy.
    Returns the MinkowskiSpace object.
    """
    space = MinkowskiSpace()
    print(f"--- Simulating Embedding of {num_fib_events} Fibonacci Events ---")

    # 1. Create events and add to space
    for i in range(num_fib_events):
        fib_val = get_fibonacci_number(i)
        event = FibonacciEvent(i, fib_val)
        space.add_event(event)
        # print(f"Added: {event}")

    # 2. Iteratively enforce causality
    # This is a simplification. The paper's algorithm is more complex for convergence.
    print(f"\n--- Enforcing Causality for {num_enforcement_iterations} Iterations ---")
    for iteration in range(num_enforcement_iterations):
        adjustments_made = False
        # Iterate over potential child-parent pairs (N-1 -> N, N-2 -> N)
        for i in range(2, num_fib_events): # F_2 onwards
            child_event = space.get_event(i)

            # Enforce F_{i-1} -> F_i
            parent1_event = space.get_event(i - 1)
            if parent1_event:
                if space.enforce_causality_step(parent1_event, child_event):
                    adjustments_made = True

            # Enforce F_{i-2} -> F_i (this might be implicitly handled by F_{i-1} -> F_i and F_{i-2} -> F_{i-1})
            # The paper's algorithm for multiple parents is more nuanced.
            # For simplicity, we primarily enforce the direct F_{N-1} -> F_N path.
            # A full implementation would need to consider all parent-child pairs for adjustment.

        if not adjustments_made and iteration > 0:
            # print(f"No adjustments made in iteration {iteration}. Early convergence.")
            break
        # if iteration % 50 == 0 or iteration == num_enforcement_iterations - 1:
        #     print(f"Iteration {iteration}: Adjustments made = {adjustments_made}")

    print("\n--- Embedding Simulation Complete ---")
    return space

# --- 5. Find Next Fibonacci Randomly ---
def find_next_fibonacci_randomly(space: MinkowskiSpace, start_index: int):
    """
    Starts at a given index, finds its parents via causal query,
    and then calculates the next Fibonacci number.
    This simulates the "randomly find next" by picking a starting point.
    """
    start_time = time.time()
    print(f"Starting causal query at F_{start_index} at {start_time}")
    current_event = space.get_event(start_index)
    if not current_event:
        print(f"Event for F_{start_index} not found in space.")
        return None


    print(f"\n--- Finding Next Fibonacci from F_{start_index} ---")
    print(f"Current event: {current_event}")

    parents = space.find_parents_by_causality(current_event)

    if len(parents) < 2:
        print(f"Could not find two causal parents for F_{start_index} in the embedding. Cannot calculate next.")
        # Fallback to direct calculation if not enough parents are found in embedding
        if start_index >= 2:
            val_n_minus_1 = get_fibonacci_number(start_index - 1)
            val_n_minus_2 = get_fibonacci_number(start_index - 2)
            next_fib_val = current_event.value + val_n_minus_1 # F_N + F_{N-1} is not F_{N+1}
            # Correct next Fibonacci is F_{N+1} = F_N + F_{N-1}
            # Or if finding F_N from F_{N-1} and F_{N-2}: F_N = F_{N-1} + F_{N-2}
            # The request was "calculate next Fibonacci number randomly"
            # If current_event is F_N, the next is F_{N+1} = F_N + F_{N-1}
            # So we need F_N and F_{N-1}.

            # Let's assume 'parents' are F_{N-1} and F_{N-2} for current_event F_N
            # If we want F_{N+1}, we need F_N (current_event.value) and F_{N-1} (one of the parents)
            parent_n_minus_1 = None
            for p in parents:
                if p.index == start_index - 1:
                    parent_n_minus_1 = p
                    break

            if parent_n_minus_1:
                next_fib_value = current_event.value + parent_n_minus_1.value
                print(f"Identified parents (by index): {parent_n_minus_1.index} and {current_event.index}")
                print(f"Calculated next Fibonacci value (F_{start_index+1}): {next_fib_value}")
                print(f"Time taken: {time.time() - start_time} seconds")
                return next_fib_value
            else:
                print("Could not identify F_{N-1} from causal parents to calculate next.")
                return None
        else:
            print("Cannot calculate next Fibonacci for very small indices.")
            return None
    else:
        # Assuming parents[0] is F_{N-1} and parents[1] is F_{N-2} by sorting
        # (or just pick the one with index N-1)
        parent_n_minus_1_val = None
        for p in parents:
            if p.index == start_index - 1:
                parent_n_minus_1_val = p.value
                break

        if parent_n_minus_1_val is not None:
            next_fib_value = current_event.value + parent_n_minus_1_val
            print(f"Identified parents (by causal query): {', '.join([str(p.index) for p in parents])}")
            print(f"Calculated next Fibonacci value (F_{start_index+1}): {next_fib_value}")
            print(f"Time taken: {time.time() - start_time} seconds")
            return next_fib_value
        else:
            print("Could not find F_{N-1} among causal parents to calculate next.")
            return None

# --- 6. A* Search Implementation ---
def a_star_search(start_index, goal_index, space: MinkowskiSpace):
    """
    Performs A* search on Fibonacci indices, using spacetime distance as heuristic.
    Nodes are Fibonacci indices. Edges are (N -> N+1) and (N -> N+2).
    """
    print(f"\n--- Performing A* Search from F_{start_index} to F_{goal_index} ---")

    start_event = space.get_event(start_index)
    goal_event = space.get_event(goal_index)

    if not start_event or not goal_event:
        print("Start or Goal event not found in space.")
        return None

    # Priority queue: (f_score, current_index)
    open_set = [(0, start_index)] # (f_score, index)

    # g_score[index] is the cost from start to index
    g_score = {start_index: 0}

    # f_score[index] = g_score[index] + h(index)
    f_score = {start_index: space.get_event(start_index).t - goal_event.t} # Simple time difference heuristic

    # came_from[index] is the index of the event preceding current_index on the cheapest path
    came_from = {}

    while open_set:
        current_f_score, current_index = heapq.heappop(open_set)

        if current_index == goal_index:
            path = []
            while current_index in came_from:
                path.append(current_index)
                current_index = came_from[current_index]
            path.append(start_index)
            return path[::-1] # Reverse to get path from start to goal

        current_event = space.get_event(current_index)

        # Define possible "neighbors" (next Fibonacci indices)
        # We can move from F_N to F_{N+1} or F_{N+2}
        neighbors_indices = []
        if current_index + 1 <= goal_index: # Only explore forward towards goal
             neighbors_indices.append(current_index + 1)
        if current_index + 2 <= goal_index: # Only explore forward towards goal
             neighbors_indices.append(current_index + 2)

        for neighbor_index in neighbors_indices:
            neighbor_event = space.get_event(neighbor_index)
            if not neighbor_event:
                continue # Neighbor not in our embedded space

            # Cost to reach neighbor (each step costs 1)
            tentative_g_score = g_score[current_index] + 1

            if tentative_g_score < g_score.get(neighbor_index, float('inf')):
                came_from[neighbor_index] = current_index
                g_score[neighbor_index] = tentative_g_score

                # Heuristic: Euclidean spatial distance to goal event
                # The paper emphasizes causality, not distance, for hierarchy.
                # But for A*, we need a distance-based heuristic.
                # Let's use the Euclidean distance in (x,y,z) space as heuristic.
                dx_h = neighbor_event.x - goal_event.x
                dy_h = neighbor_event.y - goal_event.y
                dz_h = neighbor_event.z - goal_event.z
                h_score = math.sqrt(dx_h**2 + dy_h**2 + dz_h**2)

                f_score[neighbor_index] = tentative_g_score + h_score
                heapq.heappush(open_set, (f_score[neighbor_index], neighbor_index))

    return None # No path found


# 1. Simulate embedding Fibonacci indices into Minkowski Spacetime
# We'll embed up to F_20 for demonstration
num_events_to_embed = 100 
minkowski_space = simulate_fibonacci_embedding(num_events_to_embed, num_enforcement_iterations=1000)

# Print some final event coordinates after adjustments
print("\n--- Final Event Coordinates (Sample) ---")
for i in range(0, num_events_to_embed, 5):
    event = minkowski_space.get_event(i)
    if event:
        print(event)

# 2. Demonstrate finding the next Fibonacci number randomly (by causal query)
# Pick a random starting index (e.g., between 5 and 15)
random_start_fib_index = random.randint(5, num_events_to_embed - 2) # Ensure parents exist
found_next_fib_val = find_next_fibonacci_randomly(minkowski_space, random_start_fib_index)
start_time = time.time()
if found_next_fib_val is not None:
    print(f"Confirmed F_{random_start_fib_index+1} is {found_next_fib_val}")
    print(f"Actual F_{random_start_fib_index+1} is {get_fibonacci_number(random_start_fib_index+1)}")
    print(f"Success: !! {found_next_fib_val == get_fibonacci_number(random_start_fib_index+1)}")
    print(f"Time taken: {time.time() - start_time} seconds")
# 3. Perform A* search
start_a_star_index = 3
goal_a_star_index = 15
path = a_star_search(start_a_star_index, goal_a_star_index, minkowski_space)

if path:
    print(f"\n--- A* Path from F_{start_a_star_index} to F_{goal_a_star_index} ---")
    print(" -> ".join([f"F_{idx}" for idx in path]))
    print("Actual Fibonacci values along path:")
    print(" -> ".join([str(get_fibonacci_number(idx)) for idx in path]))
else:
    print(f"\nNo A* path found from F_{start_a_star_index} to F_{goal_a_star_index}.")



--- Simulating Embedding of 100 Fibonacci Events ---

--- Enforcing Causality for 1000 Iterations ---

--- Embedding Simulation Complete ---

--- Final Event Coordinates (Sample) ---
E(0, F=0, t=0.00, x=0.17, y=-0.60, z=-0.19)
E(5, F=5, t=5.88, x=-0.22, y=0.24, z=-0.98)
E(10, F=55, t=11.92, x=0.78, y=-0.79, z=-0.92)
E(15, F=610, t=17.41, x=0.71, y=-0.35, z=0.86)
E(20, F=6765, t=22.64, x=-0.30, y=0.27, z=-0.79)
E(25, F=75025, t=28.47, x=-0.37, y=0.45, z=1.00)
E(30, F=832040, t=33.98, x=0.81, y=-0.52, z=-0.94)
E(35, F=9227465, t=41.55, x=0.97, y=-0.76, z=-0.31)
E(40, F=102334155, t=45.91, x=0.24, y=0.92, z=-0.68)
E(45, F=1134903170, t=51.57, x=-0.68, y=0.98, z=0.63)
E(50, F=12586269025, t=57.02, x=0.67, y=0.66, z=-0.51)
E(55, F=139583862445, t=61.06, x=0.98, y=0.43, z=-0.44)
E(60, F=1548008755920, t=67.89, x=0.02, y=-0.62, z=-0.12)
E(65, F=17167680177565, t=75.33, x=-0.89, y=0.99, z=0.31)
E(70, F=190392490709135, t=84.14, x=-0.88, y=0.46, z=0.70)
E(75, F=2111485077978050, t=90.11, x=-0.8